<a href="https://www.kaggle.com/code/dalloliogm/blend-of-top-4-public-solutions-multiple-strateg?scriptVersionId=246485374" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Blend of top solutions


This notebook is based on the amazing notebook https://www.kaggle.com/code/nina2025/blend-of-top-3-public-solutions . Thanks for sharing it!

I've added a few additional strategies for combining the submissions.

# Update

Based the notebook on this one instead: https://www.kaggle.com/code/nina2025/ensemble-of-top-4-public-solutions Thanks @nina2025 for sharing it!



In [1]:
import pandas as pd
from collections import Counter

# Load new predictions
path = '/kaggle/input/19-june-2025-fertilizer/'

# Define model names based on filenames
file_map = {
    'LB_0': 'LB__0_37_474',
    'LB_1': 'LB__0_37_631',
    'LB_2': 'LB__0_37_780',
    'LB_3': 'LB__0_37_856',
    'GEN_1': 'LB__0_GEN_1',
    'GEN_2': 'LB__0_GEN_2',
    'GEN_3': 'LB__0_GEN_3',
    'GEN_4': 'LB__0_GEN_4',
    'GEN_5': 'LB__0_GEN_5',
}


In [2]:
# Read and rename
dfs = []
for name, fname in file_map.items():
    df = pd.read_csv(path + f'submission__{fname}.csv')
    df = df.rename(columns={'Fertilizer Name': name})
    dfs.append(df)

# Merge all on 'id'
from functools import reduce
df_merged = reduce(lambda left, right: pd.merge(left, right, on='id'), dfs)

# Utility function
def get_top3(row, method='vote', weights=None):
    preds = [row[name] for name in file_map.keys()]
    if method == 'vote':
        return ' '.join([x for x, _ in Counter(preds).most_common(3)])

    elif method == 'weighted':
        score_map = {}
        for name in file_map.keys():
            label = row[name]
            score_map[label] = score_map.get(label, 0) + weights.get(name, 1.0)
        sorted_preds = sorted(score_map.items(), key=lambda x: -x[1])
        seen, result = set(), []
        for label, _ in sorted_preds:
            if label not in seen:
                result.append(label)
                seen.add(label)
            if len(result) == 3:
                break
        return ' '.join(result)

    elif method == 'consensus_fill':
        counts = Counter(preds)
        common = [x for x, c in counts.items() if c > 1]
        rest = [x for x in preds if x not in common]
        final = common + [x for x in rest if x not in common]
        return ' '.join(dict.fromkeys(final))[:3]

    elif method == 'stacked':
        return ' '.join(dict.fromkeys(preds))[:3]

    elif method == 'diverse':
        return ' '.join(dict.fromkeys(preds))[:3]

    else:
        raise ValueError("Unknown method")

# Define weights manually or based on LB (optional guess)
weights = {
    'LB_0': 0.92,
    'LB_1': 0.89,
    'LB_2': 0.88,
    'LB_3': 0.85,
    'GEN_1': 0.80,
    'GEN_2': 0.78,
    'GEN_3': 0.76,
    'GEN_4': 0.75,
    'GEN_5': 0.74
}

# Run ensemble and export multiple versions
methods = ['vote', 'weighted', 'consensus_fill', 'stacked', 'diverse']

for method in methods:
    df_merged['Fertilizer Name'] = df_merged.apply(get_top3, axis=1, method=method, weights=weights)
    sub = df_merged[['id', 'Fertilizer Name']]
    sub.to_csv(f'ensemble_{method}.csv', index=False)